In [ ]:
import os
import torch
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# device = "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

In [ ]:


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}

print(training_data)
# = torch.Size([1, 28, 28])

train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


# figure = plt.figure(figsize=(8, 8))
# cols, rows = 3, 3
# for i in range(1, cols * rows + 1):
#     sample_idx = torch.randint(len(training_data), size=(1,)).item()
#     img, label = training_data[sample_idx]
#     figure.add_subplot(rows, cols, i)
#     plt.title(labels_map[label])
#     plt.axis("off")
#     plt.imshow(img.squeeze(), cmap="gray")
# plt.show()


In [ ]:



class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )


    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

model = NeuralNet().to(device)
# print(model)



In [ ]:

learning_rate = 0.00005

cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        # print(images.shape)
        # print(images.view(-1, 28*28).shape)
        logits = model(images.view(-1, 28*28))
        loss = cross_entropy_loss(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


# X = torch.rand(1, 28, 28, device=device)
# logits = model(X)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")

In [ ]:

correct = 0
total = 0

with torch.no_grad():
    for test_batch in test_loader:
        images, labels = test_batch
        images, labels = images.to(device), labels.to(device)

        outputs = model(images.view(-1, 28*28))
        for idx, output in enumerate(outputs):
            if torch.argmax(output) == labels[idx]:
                correct += 1
            total += 1

print(f'accuracy: {round(correct/total, 3)}')

In [ ]:
import random

figure = plt.figure(figsize=(15, 15))
cols, rows = 5, 5
# figure = plt.figure(figsize=(8, 8))

random_indices = random.sample(range(len(test_data)), cols * rows)

for i, random_index in enumerate(random_indices):
    image, label = test_data[random_index]
    image_for_model = image.unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image_for_model)
        predicted_label = output.argmax(1).item()
    
    # Visualize
    figure.add_subplot(rows, cols, i + 1)
    plt.title(f"True: {labels_map[label]}\n Predicted: {labels_map[predicted_label]}")
    plt.axis("off")
    plt.imshow(image.squeeze(), cmap="gray")

plt.show()
# plt.savefig("final.png", dpi=300, bbox_inches="tight")


In [ ]:


torch.save(model.state_dict(), "model_state.pth")